In [ ]:
import cv2
import time
import numpy as np
import handtrackingModule2 as htm
import math
import subprocess
import os

def set_brightness(bright):
  if(0<=bright<=1):
    subprocess.run(["brightness", str(bright)])


def hand_to_bright(length,min_len=50,max_len=500):
  min_light=0
  max_light=1
  bright=np.interp(length,[min_len,max_len],[min_light,max_light])
  return bright

wCam,hCam=1280,720

cap=cv2.VideoCapture(0)
cap.set(3,wCam)
cap.set(4,hCam)
pTime=0
brightbar=0
detector=htm.handDetector(detectionCon=0.7)


while True:
    success,img=cap.read()
    img=detector.findHands(img)
    lmlist=detector.findPosition(img,draw=False)
    cv2.rectangle(img,(100,500),(50,50),(255,0,0),3)
    if len(lmlist)!=0:
      #print(lmlist[4],lmlist[8])

      x1,y1=lmlist[4][1],lmlist[4][2]   
      x2,y2=lmlist[8][1],lmlist[8][2]

      cx,cy=(x1+x2)//2,(y1+y2)//2
      cv2.circle(img,(x1,y1),15,(255,0,255),cv2.FILLED)
      cv2.circle(img,(x2,y2),15,(255,0,255),cv2.FILLED)
      cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)
      cv2.circle(img,(cx,cy),15,(255,0,255),cv2.FILLED)

      length=math.hypot(x2-x1,y2-y1)
      brightbar=np.interp(length,[50,500],[500,50])
      ratio=(int(brightbar)-50)/(500-50)
      color=(0,int(255*(1-ratio)),int(255*ratio))
      cv2.rectangle(img,(100,500),(50,int(brightbar)),color,cv2.FILLED)
      
      if(length<50):
       cv2.circle(img,(cx,cy),15,(0,255,0),cv2.FILLED)
      
      len1=hand_to_bright(length)
      set_brightness(len1)



    cTime=time.time()
    fps=1/(cTime-pTime)
    pTime=cTime

    cv2.putText(img,f'FPS:{int(fps)}',(150,70),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),3)

    cv2.imshow("Img",img)
    cv2.waitKey(1)